In [1]:
#Import Required Libraries
import keras # ML Model library
import numpy as np #linear algebra
import pandas as pd #Data processing, CSV file I/O (e.g. pd.read_csv)
import xarray as xr #Data Storage as N-D Dataset
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical
from matplotlib import pyplot


#Pre-run configuration
np.random.seed(56)

#Model Configuration
image_width = 28
image_height = 28
batch_size = 300
no_epochs = 10
no_classes = 10
validation_split = 0.2
verbosity = 1
base_filters = 128


Using TensorFlow backend.
C:\Users\Jaytwo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jaytwo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jaytwo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jaytwo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

In [2]:
#Import Training and Test datasets
train = pd.read_csv("../input/digit-recognizer/train.csv")
finaltest = pd.read_csv("../input/digit-recognizer/test.csv")
train['label'].value_counts(normalize=False)

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [3]:
#Prepare Datalists
trainimages = []
trainlabels = []

In [4]:
train.values

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [7, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [5]:
#Define Ingest Function
def ingest_data(itr, data):
    temptrain = data
    #Split Labels
    templabels = temptrain['label']
    #Convert Labels to bool
    templabels = np.where(templabels == itr, 1, 0)
    templabels = to_categorical(templabels)
    #Split Images
    tempimages = temptrain.drop(['label'], axis = 1)
    tempimages = tempimages.values.reshape(-1, image_width, image_height, 1)
    tempimages = tempimages.astype('float32')
    tempimages = tempimages/255
    return templabels, tempimages
    
    

In [6]:
for i in range(no_classes):
    temp = ingest_data(i, train)
    trainlabels.append(temp[0])
    trainimages.append(temp[1])
del train

In [7]:
trainimages[0].shape



(42000, 28, 28, 1)

In [8]:
trainlabels[0]

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [9]:
#Train/Test Split
x_train, x_test, y_train, y_test = train_test_split(trainimages, trainlabels, test_size = validation_split)

In [10]:
#Verify the data shape
x_train[0].shape

(42000, 28, 28, 1)

In [11]:
#Verify label shape
y_train[0].shape

(42000, 2)

In [12]:
#Define the models
models = list()
for i in range(no_classes):
    model = Sequential()
    model.add(Conv2D(filters = (int(base_filters/4)), kernel_size = (7,7), padding = 'Same', activation = 'relu', batch_input_shape = (42000, image_width, image_height, 1)))
    model.add(Conv2D(filters = (int(base_filters/4), kernel_size = (7,7), padding = 'Same', activation = 'relu'))
    #model.add(Conv2D(filters = (base_filters/2), kernel_size = (5,5), padding = 'Same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = (base_filters/2), kernel_size = (5,5), padding = 'Same', activation = 'relu'))
    #model.add(Conv2D(filters = (base_filters/2), kernel_size = (4,4), padding = 'Same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = (base_filters), kernel_size = (3,3), padding = 'Same', activation = 'relu'))
    #model.add(Conv2D(filters = (base_filters), kernel_size = (2,2), padding = 'Same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(16, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = "softmax"))
    models.insert(i, model)
models[0].summary()

Instructions for updating:
Colocations handled automatically by placer.


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [13]:
#Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [14]:
#Compile the models
for i in range(len(models)):
    models[i].compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [15]:
#Set LRR
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.00001)

In [18]:
#train the individual models for each number
histories = list()
for i in range(len(models)):
    histories.insert(i, models[i].fit(
        x_train[i], y_train[i], 
        batch_size = batch_size, epochs = no_epochs,verbose = 1, 
        validation_data = (x_test[i], y_test[i]), callbacks = [learning_rate_reduction]
    ))

In [17]:
pyplot.plot(histories[0].history['accuracy'], label='Training Accuracy')
pyplot.plot(histories[0].history['val_accuracy'], label='Validation Accuracy')

IndexError: list index out of range